#### Compare restaurant distributions across UberEats, Deliveroo, and Takeaway. What are some of the market trends?

##### preparing database settings

In [6]:
#prepare database connections

import sqlite3
import pandas as pd

# Paths to your databases
deliveroo_db_path = "C:\\repos\Project-SQL\\databases\\deliveroo.db"
ubereats_db_path = "C:\\repos\Project-SQL\\databases\\ubereats.db"
takeaway_db_path = "C:\\repos\Project-SQL\\databases\\takeaway.db"

# Connect to the Deliveroo database
conn = sqlite3.connect(deliveroo_db_path)

# Attach the other databases
conn.execute(f"ATTACH DATABASE '{ubereats_db_path}' AS ubereats_db;")
conn.execute(f"ATTACH DATABASE '{takeaway_db_path}' AS takeaway_db;")

# Check attached databases
cursor = conn.cursor()  # Initialize the cursor
cursor.execute("PRAGMA database_list;")
print("Attached Databases:")
for row in cursor.fetchall():
    print(row)


Attached Databases:
(0, 'main', 'C:\\repos\\Project-SQL\\databases\\deliveroo.db')
(2, 'ubereats_db', 'C:\\repos\\Project-SQL\\databases\\ubereats.db')
(3, 'takeaway_db', 'C:\\repos\\Project-SQL\\databases\\takeaway.db')


##### Executing query

In [7]:
# Query to execute
query = """
SELECT 'Deliveroo' AS platform, COUNT(*) AS restaurant_count
FROM main.restaurants

UNION ALL

SELECT 'UberEats' AS platform, COUNT(*) AS restaurant_count
FROM ubereats_db.restaurants

UNION ALL

SELECT 'Takeaway' AS platform, COUNT(*) AS restaurant_count
FROM takeaway_db.restaurants; 
"""

# Execute the query
cursor = conn.cursor()
cursor.execute(query)

# Fetch the results
columns = [description[0] for description in cursor.description]  # Extract column names
results = cursor.fetchall()  # Fetch all rows

# Convert to a pandas DataFrame
df = pd.DataFrame(results, columns=columns)

# Display the DataFrame
df  # Display as output in the notebook


,platform,restaurant_count
0,Deliveroo,2716
1,UberEats,2729
2,Takeaway,4071


In [8]:
query2="""
SELECT 'Deliveroo' AS platform, COUNT(*) AS total_restaurants
FROM main.restaurants
UNION ALL
SELECT 'UberEats' AS platform, COUNT(*) AS total_restaurants
FROM ubereats_db.restaurants
UNION ALL
SELECT 'Takeaway' AS platform, COUNT(*) AS total_restaurants
FROM takeaway_db.restaurants;

SELECT d.name AS restaurant_name, 'Deliveroo' AS platform
FROM main.restaurants AS d
WHERE EXISTS (
    SELECT 1
    FROM ubereats_db.restaurants AS u
    WHERE u.title = d.name
)
UNION ALL
SELECT u.title AS restaurant_name, 'UberEats' AS platform
FROM ubereats_db.restaurants AS u
WHERE EXISTS (
    SELECT 1
    FROM takeaway_db.restaurants AS t
    WHERE t.name = u.title
)
UNION ALL
SELECT t.name AS restaurant_name, 'Takeaway' AS platform
FROM takeaway_db.restaurants AS t
WHERE EXISTS (
    SELECT 1
    FROM main.restaurants AS d
    WHERE d.name = t.name
);
"""

# Execute the query
cursor = conn.cursor()
cursor.execute(query2)

# Fetch the results
columns = [description[0] for description in cursor.description]  # Extract column names
results = cursor.fetchall()  # Fetch all rows

# Convert to a pandas DataFrame
df = pd.DataFrame(results, columns=columns)

# Display the DataFrame
df  # Display as output in the notebook

ProgrammingError: You can only execute one statement at a time.